# Hodina 11: Prostor stavů - Definice a modelování problémů 🎯

## Přehled lekce

V této hodině se naučíme:
- Co je prostor stavů a jak modelovat problémy
- Jak reprezentovat stavy pomocí neuronových sítí
- Praktické implementace s transformery
- Vizualizace a interaktivní explorování stavových prostorů

---

## 1. Nastavení prostředí a instalace knihoven

In [ ]:
# Instalace potřebných knihoven
!pip install torch torchvision transformers gradio networkx matplotlib numpy pandas
!pip install plotly ipywidgets

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import deque, defaultdict
import heapq
from IPython.display import HTML, display
import plotly.graph_objects as go
import gradio as gr
from transformers import pipeline
import json
import time

# Nastavení zobrazení
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
torch.manual_seed(42)

## 2. Co je prostor stavů? 🌐

Prostor stavů je fundamentální koncept v AI pro reprezentaci a řešení problémů.

In [ ]:
# Interaktivní vizualizace konceptu prostoru stavů
class StateSpaceVisualizer:
    def __init__(self):
        self.states = []
        self.transitions = []
    
    def add_state(self, name, properties):
        self.states.append({'name': name, 'properties': properties})
    
    def add_transition(self, from_state, to_state, action):
        self.transitions.append({
            'from': from_state,
            'to': to_state,
            'action': action
        })
    
    def visualize_3d(self):
        # Vytvoření 3D grafu stavového prostoru
        fig = go.Figure()
        
        # Pozice uzlů
        n = len(self.states)
        pos = {}
        for i, state in enumerate(self.states):
            angle = 2 * np.pi * i / n
            pos[state['name']] = [
                np.cos(angle) * 2,
                np.sin(angle) * 2,
                np.random.uniform(-1, 1)
            ]
        
        # Přidání hran
        for trans in self.transitions:
            x0, y0, z0 = pos[trans['from']]
            x1, y1, z1 = pos[trans['to']]
            
            fig.add_trace(go.Scatter3d(
                x=[x0, x1], y=[y0, y1], z=[z0, z1],
                mode='lines',
                line=dict(color='lightblue', width=3),
                hoverinfo='text',
                text=f"Akce: {trans['action']}",
                showlegend=False
            ))
        
        # Přidání uzlů
        for state in self.states:
            x, y, z = pos[state['name']]
            
            fig.add_trace(go.Scatter3d(
                x=[x], y=[y], z=[z],
                mode='markers+text',
                marker=dict(size=20, color='red'),
                text=state['name'],
                textposition='top center',
                hoverinfo='text',
                hovertext=f"{state['name']}\n{state['properties']}",
                showlegend=False
            ))
        
        fig.update_layout(
            title="3D Vizualizace prostoru stavů",
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            height=600
        )
        
        return fig

# Příklad: Prostor stavů pro navigaci robota
visualizer = StateSpaceVisualizer()

# Definice stavů
states = [
    ('Start', 'pozice: (0,0), energie: 100%'),
    ('Křižovatka', 'pozice: (2,2), energie: 80%'),
    ('Nabíječka', 'pozice: (3,1), energie: 100%'),
    ('Překážka', 'pozice: (1,3), energie: 70%'),
    ('Cíl', 'pozice: (4,4), energie: 60%')
]

for name, props in states:
    visualizer.add_state(name, props)

# Definice přechodů
transitions = [
    ('Start', 'Křižovatka', 'Jít vpřed'),
    ('Křižovatka', 'Nabíječka', 'Otočit vpravo'),
    ('Křižovatka', 'Překážka', 'Otočit vlevo'),
    ('Nabíječka', 'Cíl', 'Jít vpřed'),
    ('Překážka', 'Křižovatka', 'Obejít'),
    ('Křižovatka', 'Cíl', 'Jít rovně')
]

for from_s, to_s, action in transitions:
    visualizer.add_transition(from_s, to_s, action)

# Zobrazení
fig = visualizer.visualize_3d()
fig.show()

print("🤖 Prostor stavů reprezentuje všechny možné stavy a přechody v problému!")

## 3. Neuronová reprezentace stavů 🧠

Použijeme neuronové sítě pro učení reprezentací stavů.

In [ ]:
# Neuronový enkodér stavů
class StateEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(StateEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, latent_dim)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def encode(self, x):
        return self.encoder(x)
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        z = self.encode(x)
        x_reconstructed = self.decode(z)
        return x_reconstructed, z

# Trénování enkodéru na příkladu šachových pozic
def generate_chess_states(n_samples=1000):
    """Generuje zjednodušené šachové stavy"""
    states = []
    labels = []
    
    for _ in range(n_samples):
        # 8x8 šachovnice, každé pole může mít 0 (prázdné) nebo 1-6 (různé figury)
        board = np.random.randint(0, 7, size=64)
        
        # Jednoduchá heuristika pro ohodnocení pozice
        material_value = np.sum(board) / 64
        center_control = np.sum(board.reshape(8, 8)[3:5, 3:5]) / 4
        
        states.append(board)
        labels.append(material_value + center_control)
    
    return torch.FloatTensor(states), torch.FloatTensor(labels)

# Vytvoření a trénování modelu
states, values = generate_chess_states()
model = StateEncoder(input_dim=64, hidden_dim=128, latent_dim=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Trénování
print("🎯 Trénování neuronového enkodéru stavů...")
losses = []

for epoch in range(100):
    optimizer.zero_grad()
    reconstructed, latent = model(states)
    loss = criterion(reconstructed, states)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 20 == 0:
        print(f"Epocha {epoch}: Loss = {loss.item():.4f}")

# Vizualizace latentního prostoru
with torch.no_grad():
    latent_representations = model.encode(states).numpy()

# PCA pro 2D vizualizaci
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
latent_2d = pca.fit_transform(latent_representations)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], 
                     c=values.numpy(), cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Hodnota pozice')
plt.title('Latentní prostor šachových pozic')
plt.xlabel('První komponenta')
plt.ylabel('Druhá komponenta')
plt.show()

print("\n✅ Model se naučil komprimovat 64D stavy do 16D latentního prostoru!")

## 4. Modelování problémů jako prostor stavů 🔧

Implementujeme různé typy problémů jako prostory stavů.

In [ ]:
# Abstraktní třída pro problém
class Problem:
    def __init__(self):
        self.initial_state = None
        self.goal_state = None
    
    def actions(self, state):
        """Vrátí možné akce ze stavu"""
        raise NotImplementedError
    
    def result(self, state, action):
        """Vrátí nový stav po provedení akce"""
        raise NotImplementedError
    
    def is_goal(self, state):
        """Kontroluje, zda je stav cílový"""
        raise NotImplementedError
    
    def path_cost(self, path):
        """Vypočítá cenu cesty"""
        return len(path)

# Příklad 1: 8-puzzle s neuronovým hodnocením
class NeuralPuzzle8(Problem):
    def __init__(self):
        super().__init__()
        self.size = 3
        self.initial_state = self.generate_random_state()
        self.goal_state = tuple(range(9))  # 0,1,2,3,4,5,6,7,8
        
        # Neuronová síť pro hodnocení stavů
        self.value_network = nn.Sequential(
            nn.Linear(9, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def generate_random_state(self):
        state = list(range(9))
        np.random.shuffle(state)
        return tuple(state)
    
    def actions(self, state):
        """Možné pohyby prázdného políčka (0)"""
        state = list(state)
        empty_idx = state.index(0)
        row, col = empty_idx // 3, empty_idx % 3
        
        actions = []
        if row > 0: actions.append('UP')
        if row < 2: actions.append('DOWN')
        if col > 0: actions.append('LEFT')
        if col < 2: actions.append('RIGHT')
        
        return actions
    
    def result(self, state, action):
        state = list(state)
        empty_idx = state.index(0)
        row, col = empty_idx // 3, empty_idx % 3
        
        if action == 'UP' and row > 0:
            swap_idx = (row - 1) * 3 + col
        elif action == 'DOWN' and row < 2:
            swap_idx = (row + 1) * 3 + col
        elif action == 'LEFT' and col > 0:
            swap_idx = row * 3 + (col - 1)
        elif action == 'RIGHT' and col < 2:
            swap_idx = row * 3 + (col + 1)
        else:
            return tuple(state)
        
        state[empty_idx], state[swap_idx] = state[swap_idx], state[empty_idx]
        return tuple(state)
    
    def is_goal(self, state):
        return state == self.goal_state
    
    def heuristic(self, state):
        """Neuronová heuristika"""
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            value = self.value_network(state_tensor).item()
        return abs(value)
    
    def visualize_state(self, state):
        """Vizualizace stavu puzzle"""
        state = list(state)
        grid = np.array(state).reshape(3, 3)
        
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.matshow(grid != 0, cmap='RdBu')
        
        for i in range(3):
            for j in range(3):
                val = grid[i, j]
                if val != 0:
                    ax.text(j, i, str(val), ha='center', va='center', 
                           fontsize=20, fontweight='bold')
        
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title('8-Puzzle stav')
        plt.show()

# Vytvoření a vizualizace problému
puzzle = NeuralPuzzle8()
print("🧩 8-Puzzle problém:")
print(f"Počáteční stav: {puzzle.initial_state}")
print(f"Cílový stav: {puzzle.goal_state}")
print(f"Možné akce: {puzzle.actions(puzzle.initial_state)}")

puzzle.visualize_state(puzzle.initial_state)

## 5. Transformery pro plánování v prostoru stavů 🤖

Použijeme transformery pro učení se plánovat v prostoru stavů.

In [ ]:
# Transformer pro plánování akcí
class TransformerPlanner(nn.Module):
    def __init__(self, state_dim, action_dim, d_model=128, n_heads=4, n_layers=3):
        super(TransformerPlanner, self).__init__()
        
        self.state_embedding = nn.Linear(state_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.randn(1, 100, d_model))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            batch_first=True
        )
        
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.action_head = nn.Linear(d_model, action_dim)
        self.value_head = nn.Linear(d_model, 1)
    
    def forward(self, states_sequence):
        # states_sequence: [batch, seq_len, state_dim]
        seq_len = states_sequence.size(1)
        
        # Embed states
        embedded = self.state_embedding(states_sequence)
        embedded += self.positional_encoding[:, :seq_len, :]
        
        # Transformer processing
        transformed = self.transformer(embedded)
        
        # Get action probabilities and value
        action_logits = self.action_head(transformed)
        values = self.value_head(transformed)
        
        return action_logits, values

# Příklad: Navigační úloha s transformerem
class NavigationEnvironment:
    def __init__(self, grid_size=10):
        self.grid_size = grid_size
        self.reset()
    
    def reset(self):
        self.agent_pos = [0, 0]
        self.goal_pos = [self.grid_size-1, self.grid_size-1]
        self.obstacles = self._generate_obstacles()
        return self.get_state()
    
    def _generate_obstacles(self, n_obstacles=15):
        obstacles = set()
        for _ in range(n_obstacles):
            x = np.random.randint(1, self.grid_size-1)
            y = np.random.randint(1, self.grid_size-1)
            if (x, y) != tuple(self.goal_pos):
                obstacles.add((x, y))
        return obstacles
    
    def get_state(self):
        # One-hot encoding of agent position + goal + obstacles
        state = np.zeros((self.grid_size, self.grid_size, 3))
        state[self.agent_pos[0], self.agent_pos[1], 0] = 1  # Agent
        state[self.goal_pos[0], self.goal_pos[1], 1] = 1    # Goal
        for obs in self.obstacles:
            state[obs[0], obs[1], 2] = 1  # Obstacles
        return state.flatten()
    
    def step(self, action):
        # Actions: 0=up, 1=right, 2=down, 3=left
        moves = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        dx, dy = moves[action]
        
        new_x = self.agent_pos[0] + dx
        new_y = self.agent_pos[1] + dy
        
        # Check boundaries and obstacles
        if (0 <= new_x < self.grid_size and 
            0 <= new_y < self.grid_size and
            (new_x, new_y) not in self.obstacles):
            self.agent_pos = [new_x, new_y]
        
        # Calculate reward
        if self.agent_pos == self.goal_pos:
            reward = 100
            done = True
        else:
            reward = -1
            done = False
        
        return self.get_state(), reward, done
    
    def render(self):
        grid = np.zeros((self.grid_size, self.grid_size))
        
        for obs in self.obstacles:
            grid[obs] = 0.5
        
        grid[tuple(self.agent_pos)] = 1
        grid[tuple(self.goal_pos)] = 0.8
        
        plt.figure(figsize=(6, 6))
        plt.imshow(grid, cmap='coolwarm')
        plt.title('Navigační prostředí')
        plt.colorbar(label='Agent=1, Cíl=0.8, Překážka=0.5')
        plt.show()

# Vytvoření a demonstrace prostředí
env = NavigationEnvironment()
state = env.reset()

print("🗺️ Navigační prostředí s transformerem:")
env.render()

# Inicializace transformer planneru
planner = TransformerPlanner(
    state_dim=300,  # 10x10x3 flattened
    action_dim=4,   # 4 směry pohybu
    d_model=64
)

print("\n🤖 Transformer planner je připraven k trénování!")

## 6. Interaktivní explorace prostoru stavů 🎮

Vytvoříme Gradio aplikaci pro interaktivní exploraci.

In [ ]:
# Gradio aplikace pro exploraci prostoru stavů
class StateSpaceExplorer:
    def __init__(self):
        self.problems = {
            "8-Puzzle": NeuralPuzzle8(),
            "Navigace": NavigationEnvironment()
        }
        self.current_problem = None
        self.search_history = []
    
    def breadth_first_search(self, problem, max_steps=1000):
        """BFS s vizualizací"""
        frontier = deque([(problem.initial_state, [])])
        explored = set()
        steps = 0
        
        while frontier and steps < max_steps:
            state, path = frontier.popleft()
            
            if problem.is_goal(state):
                return path, steps, "Cíl nalezen!"
            
            if state in explored:
                continue
            
            explored.add(state)
            
            for action in problem.actions(state):
                next_state = problem.result(state, action)
                if next_state not in explored:
                    frontier.append((next_state, path + [action]))
            
            steps += 1
        
        return [], steps, "Řešení nenalezeno"
    
    def neural_guided_search(self, problem, max_steps=1000):
        """Hledání vedené neuronovou sítí"""
        if not hasattr(problem, 'heuristic'):
            return [], 0, "Problém nepodporuje neuronové vedení"
        
        frontier = [(problem.heuristic(problem.initial_state), 
                    problem.initial_state, [])]
        explored = set()
        steps = 0
        
        while frontier and steps < max_steps:
            _, state, path = heapq.heappop(frontier)
            
            if problem.is_goal(state):
                return path, steps, "Cíl nalezen s neuronovou heuristikou!"
            
            if state in explored:
                continue
            
            explored.add(state)
            
            for action in problem.actions(state):
                next_state = problem.result(state, action)
                if next_state not in explored:
                    priority = problem.heuristic(next_state) + len(path) + 1
                    heapq.heappush(frontier, 
                                  (priority, next_state, path + [action]))
            
            steps += 1
        
        return [], steps, "Řešení nenalezeno"
    
    def explore_problem(self, problem_name, search_method):
        problem = self.problems[problem_name]
        
        if search_method == "BFS":
            path, steps, status = self.breadth_first_search(problem)
        else:
            path, steps, status = self.neural_guided_search(problem)
        
        # Vytvoření vizualizace
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Graf průběhu hledání
        ax1.plot(range(steps), np.random.random(steps).cumsum())
        ax1.set_title(f'Průběh hledání ({search_method})')
        ax1.set_xlabel('Kroky')
        ax1.set_ylabel('Prozkoumané stavy')
        
        # Statistiky
        ax2.text(0.1, 0.8, f"Metoda: {search_method}", fontsize=12)
        ax2.text(0.1, 0.6, f"Kroky: {steps}", fontsize=12)
        ax2.text(0.1, 0.4, f"Délka cesty: {len(path)}", fontsize=12)
        ax2.text(0.1, 0.2, f"Status: {status}", fontsize=12)
        ax2.axis('off')
        
        plt.tight_layout()
        plt.savefig('search_results.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        result_text = f"""
        🔍 Výsledky hledání:
        - Metoda: {search_method}
        - Počet kroků: {steps}
        - Délka nalezené cesty: {len(path)}
        - Status: {status}
        - Cesta: {' → '.join(path[:10])}{'...' if len(path) > 10 else ''}
        """
        
        return 'search_results.png', result_text

# Vytvoření Gradio rozhraní
explorer = StateSpaceExplorer()

def gradio_interface(problem_type, search_method):
    return explorer.explore_problem(problem_type, search_method)

# Gradio aplikace
demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Dropdown(
            choices=["8-Puzzle", "Navigace"],
            label="Typ problému",
            value="8-Puzzle"
        ),
        gr.Radio(
            choices=["BFS", "Neuronové vedení"],
            label="Metoda hledání",
            value="BFS"
        )
    ],
    outputs=[
        gr.Image(label="Vizualizace hledání"),
        gr.Textbox(label="Výsledky", lines=8)
    ],
    title="🎯 Explorátor prostoru stavů",
    description="Interaktivní nástroj pro exploraci různých prostorů stavů s různými metodami hledání.",
    examples=[
        ["8-Puzzle", "BFS"],
        ["8-Puzzle", "Neuronové vedení"],
        ["Navigace", "BFS"]
    ]
)

# Spuštění aplikace
print("🚀 Spouštím Gradio aplikaci...")
demo.launch(share=True)

## 7. Integrace s Ollama pro popis stavů 🦙

Použijeme Ollama pro generování přirozeného popisu stavů.

In [ ]:
# Ollama integrace pro popis stavů
print("🦙 Nastavení Ollama pro popis stavů...")

# Instalace Ollama (pro Colab)
!curl -fsSL https://ollama.com/install.sh | sh

# Stažení modelu
!ollama pull llama2:7b

# Python wrapper pro Ollama
import subprocess
import json

class OllamaStateDescriber:
    def __init__(self, model="llama2:7b"):
        self.model = model
    
    def describe_state(self, state_info):
        """Generuje přirozený popis stavu"""
        prompt = f"""
        Popiš následující stav v AI systému jednoduše a srozumitelně v češtině:
        
        Typ problému: {state_info['problem_type']}
        Aktuální stav: {state_info['current_state']}
        Možné akce: {state_info['possible_actions']}
        Vzdálenost od cíle: {state_info['distance_to_goal']}
        
        Odpověz stručně v 2-3 větách.
        """
        
        try:
            result = subprocess.run(
                ['ollama', 'run', self.model, prompt],
                capture_output=True,
                text=True,
                timeout=30
            )
            return result.stdout.strip()
        except Exception as e:
            return f"Chyba při generování popisu: {e}"
    
    def suggest_next_action(self, state_info, history):
        """Navrhuje další akci na základě historie"""
        prompt = f"""
        Jako AI expert, navrhni nejlepší další akci.
        
        Současný stav: {state_info}
        Historie akcí: {history[-5:]}
        
        Navrhni jednu akci a vysvětli proč.
        """
        
        try:
            result = subprocess.run(
                ['ollama', 'run', self.model, prompt],
                capture_output=True,
                text=True,
                timeout=30
            )
            return result.stdout.strip()
        except Exception as e:
            return f"Chyba při návrhu akce: {e}"

# Demonstrace
describer = OllamaStateDescriber()

# Příklad popisu stavu
example_state = {
    'problem_type': '8-puzzle',
    'current_state': '(1, 2, 3, 4, 0, 5, 6, 7, 8)',
    'possible_actions': ['UP', 'DOWN', 'LEFT', 'RIGHT'],
    'distance_to_goal': 5
}

print("📝 Generování popisu stavu pomocí Ollama:")
description = describer.describe_state(example_state)
print(description)

print("\n💡 Návrh další akce:")
suggestion = describer.suggest_next_action(
    example_state, 
    ['RIGHT', 'DOWN', 'LEFT']
)
print(suggestion)

## 8. Praktická cvičení 📝

Vyzkoušejte si práci s prostorem stavů!

In [ ]:
# Cvičení 1: Implementujte vlastní problém
class MyProblem(Problem):
    """Implementujte problém Hanojských věží"""
    def __init__(self, n_disks=3):
        super().__init__()
        self.n_disks = n_disks
        # TODO: Definujte počáteční a cílový stav
        # Stav může být reprezentován jako tuple tří seznamů
        pass
    
    def actions(self, state):
        # TODO: Implementujte možné tahy
        pass
    
    def result(self, state, action):
        # TODO: Implementujte provedení tahu
        pass
    
    def is_goal(self, state):
        # TODO: Kontrola cílového stavu
        pass

# Cvičení 2: Trénování neuronové heuristiky
def train_heuristic_network(problem, n_episodes=100):
    """Natrénujte síť pro odhad vzdálenosti k cíli"""
    # TODO: Implementujte trénování
    # Tip: Generujte náhodné stavy a spočítejte skutečnou vzdálenost k cíli
    pass

# Cvičení 3: Vizualizace prostoru stavů
def visualize_state_space_graph(problem, max_depth=3):
    """Vytvořte graf prostoru stavů do dané hloubky"""
    # TODO: Implementujte BFS a vytvořte NetworkX graf
    # Tip: Použijte nx.DiGraph() a přidávejte uzly a hrany
    pass

print("📚 Cvičení připravena! Implementujte řešení výše.")

## 9. Shrnutí a klíčové koncepty 🎓

### Co jsme se naučili:

1. **Prostor stavů** - fundamentální reprezentace problémů v AI
2. **Neuronové enkodéry** - komprese vysokodimenzionálních stavů
3. **Transformery pro plánování** - učení se plánovat sekvence akcí
4. **Interaktivní explorace** - vizualizace a pochopení prostorů stavů
5. **Integrace s LLM** - použití Ollama pro přirozený popis stavů

### Klíčové koncepty:
- **Stav**: Kompletní popis situace v daném okamžiku
- **Akce**: Možné přechody mezi stavy
- **Přechodová funkce**: Určuje výsledný stav po akci
- **Cílový test**: Rozpoznání dosažení cíle
- **Heuristika**: Odhad vzdálenosti k cíli

### Další kroky:
- V další hodině se podíváme na **prohledávací algoritmy**
- Naučíme se efektivně procházet prostory stavů
- Implementujeme pokročilé metody jako A* s neuronovými heuristikami

In [ ]:
# Závěrečná interaktivní ukázka
print("🎉 Gratulujeme! Dokončili jste hodinu o prostoru stavů!")
print("\n📊 Vaše pokroky:")
print("✅ Pochopení konceptu prostoru stavů")
print("✅ Implementace neuronových enkodérů")
print("✅ Práce s transformery pro plánování")
print("✅ Vytvoření interaktivních aplikací")
print("✅ Integrace s Ollama")

# Mini kvíz
def quick_quiz():
    questions = [
        {
            "q": "Co je prostor stavů?",
            "options": [
                "Množina všech možných stavů problému",
                "Pouze počáteční stav",
                "Pouze cílový stav"
            ],
            "correct": 0
        },
        {
            "q": "K čemu slouží neuronový enkodér stavů?",
            "options": [
                "K zvětšení dimenze stavů",
                "K komprimaci stavů do nižší dimenze",
                "K mazání stavů"
            ],
            "correct": 1
        }
    ]
    
    score = 0
    for i, q in enumerate(questions):
        print(f"\nOtázka {i+1}: {q['q']}")
        for j, opt in enumerate(q['options']):
            print(f"{j+1}. {opt}")
        
        # V reálné aplikaci by byla interaktivní odpověď
        print(f"Správná odpověď: {q['correct']+1}")
    
    print("\n🏆 Výborně! Jste připraveni na další hodinu!")

quick_quiz()

### Hodina 11 — Prostor stavů (ELI10)

Představ si, že řešíš hlavolam: máš start a cíl a mezi nimi různé mezistavy — to jsou "stavy". Prostor stavů je mapka všech možných stavů a jak se mezi nimi můžeš pohybovat (tyto pohyby nazýváme operátory). AI pak hledá cestu ze startu do cíle — to dělá průzkumem tohoto prostoru (např. pomocí BFS nebo DFS). V praxi to můžeme zobrazit jako graf (uzly = stavy, hrany = možné kroky). Níže je malý praktický příklad, který ukazuje, jak se v takovém prostoru hledá cesta.

In [ ]:
# Simple grid state-space + BFS and DFS example
from collections import deque

def neighbors(pos, max_r, max_c):
    r,c = pos
    for dr,dc in [(1,0),(-1,0),(0,1),(0,-1)]:
        nr,nc = r+dr, c+dc
        if 0 <= nr < max_r and 0 <= nc < max_c:
            yield (nr,nc)


def bfs(start, goal, max_r=3, max_c=3, blocked=None):
    blocked = set(blocked or [])
    q = deque([start])
    parent = {start: None}
    while q:
        cur = q.popleft()
        if cur == goal:
            # reconstruct
            path = []
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            return list(reversed(path))
        for n in neighbors(cur, max_r, max_c):
            if n in blocked or n in parent:
                continue
            parent[n] = cur
            q.append(n)
    return None


def dfs(start, goal, max_r=3, max_c=3, blocked=None):
    blocked = set(blocked or [])
    stack = [start]
    parent = {start: None}
    while stack:
        cur = stack.pop()
        if cur == goal:
            path = []
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            return list(reversed(path))
        for n in neighbors(cur, max_r, max_c):
            if n in blocked or n in parent:
                continue
            parent[n] = cur
            stack.append(n)
    return None

# Quick tests
start = (0,0)
goal = (2,2)
path_bfs = bfs(start, goal)
path_dfs = dfs(start, goal)
print('BFS path:', path_bfs)
print('DFS path (one possible):', path_dfs)
assert path_bfs is not None and path_bfs[0] == start and path_bfs[-1] == goal
assert path_dfs is not None and path_dfs[0] == start and path_dfs[-1] == goal

# Test blocked cells
blocked = [(1,0), (1,1)]
path_bfs_blocked = bfs(start, goal, blocked=blocked)
print('BFS with blocked:', path_bfs_blocked)
assert path_bfs_blocked is None or path_bfs_blocked[0] == start


Úkoly:

1) Změňte velikost gridu na 5×5 a přidejte několik blokovaných polí — porovnejte BFS vs DFS (délka cesty, pořadí průchodu).
2) Vykreslete graf prostoru stavů pomocí `networkx` a `matplotlib` (uzly = buňky, hrany = sousední kroky). 
3) Bonus: Implementujte uniform-cost search nebo A* (heuristika Manhattan distance) pro nalezení nejkratší cesty na vážené mřížce.